####calculating the points scroed by each driver to do analysis on most dominant drivers(we are doing this using sql/spark sql)
- join tables and get required cols
- add a columns total points and races by each driver 
- add a calculated points which will normalize the points like one who finished 1st will have 10, 2 -9 ...

In [0]:
dbutils.widgets.text("p_file_date","")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
# creating a table using spark.sql

spark.sql("""CREATE TABLE IF NOT EXISTS f1_presentation_db.calculated_race_results(
        race_year INT,
        team_name STRING,
        driver_id INT,
        driver_name STRING,
        race_id INT,
        position INT,
        points INT,
        calculated_points INT,
        created_date TIMESTAMP,
        updated_date TIMESTAMP
        )
        USING delta
""")


In [0]:
# creating a temporary view which which gets the data by joining some tables as per our requirement. we have added a additional condition to filter based on file date which bring only new data so that it can support incremental loads as well

spark.sql(f"""
CREATE OR REPLACE TEMP VIEW vcalculated_points 
AS
SELECT races.race_year,
constructors.name as team_name,
drivers.driver_id ,
drivers.name as driver_name,
races.race_id ,
results.position ,
results.points,
11-results.position as calculated_points
FROM f1_processed_db.results
JOIN f1_processed_db.drivers ON (results.driver_id = drivers.driver_id)
JOIN f1_processed_db.constructors ON (results.constructor_id = constructors.constructor_id)
JOIN f1_processed_db.races ON (results.race_id = races.race_id)
where results.position <= 10 
    AND results.file_date = '{v_file_date}'
""")


In [0]:
# merging the data to the target table using the view created above
from pyspark.sql.functions import current_timestamp
spark.sql("""
          merge into f1_presentation_db.calculated_race_results as t
          using vcalculated_points as s 
          on (t.driver_id = s.driver_id and t.race_id = s.race_id)
          when matched then update
          set
          t.position = s.position,
          t.points = s.points,
          t.calculated_points = s.calculated_points,
          t.updated_date = current_timestamp()
          when not matched then 
          insert (race_year,team_name,driver_id,driver_name,race_id,position,points,calculated_points,created_date,updated_date )
          values (s.race_year,s.team_name,s.driver_id,s.driver_name,s.race_id,s.position,s.points,s.calculated_points,current_timestamp(),current_timestamp())

          """)